In [17]:
# Import necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [3]:
# Load and the dataset
df = pd.read_csv('/content/judge-1377884607_tweet_product_company (1).csv',encoding = "ISO-8859-1")
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [4]:
df = df.drop(columns=['emotion_in_tweet_is_directed_at'])  # Remove unnecessary column

In [5]:
text_column = 'tweet_text'
target_column = 'is_there_an_emotion_directed_at_a_brand_or_product'

In [6]:
# Check for null values and handle them
df = df.dropna(subset=[text_column])

In [10]:
# Clean the text data
import re
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

In [11]:
# Clean text
df.loc[:,'clean_text'] = df[text_column].apply(clean_text)

In [12]:
# Lowercase the text
df.loc[:, 'clean_text'] = df['clean_text'].str.lower()

In [13]:
# Tokenization
tokenizer = Tokenizer(num_words=10000)  # Define the tokenizer with a max number of words
tokenizer.fit_on_texts(df['clean_text'])

In [14]:
# Convert text to sequences
sequences = tokenizer.texts_to_sequences(df['clean_text'])

In [15]:
# Padding sequences
maxlen = 100  # Adjust maxlen as needed
X = pad_sequences(sequences, maxlen=maxlen)
y = df[target_column].values

In [18]:
# Convert target labels to integers
y = np.array([1 if label == 'positive' else 0 for label in y])

In [20]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

LSTM

In [22]:
#build the model
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from keras.models import Sequential


model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [23]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
# Train the model
history = model.fit(X_train, y_train, batch_size=64, epochs=5, validation_data=(X_val, y_val), verbose=1)

Epoch 1/5
114/114 [==============================] - 40s 275ms/step - loss: 0.0657 - accuracy: 0.9946 - val_loss: 5.0900e-04 - val_accuracy: 1.0000
Epoch 2/5
114/114 [==============================] - 30s 263ms/step - loss: 3.5778e-04 - accuracy: 1.0000 - val_loss: 2.3776e-04 - val_accuracy: 1.0000
Epoch 3/5
114/114 [==============================] - 33s 286ms/step - loss: 1.9232e-04 - accuracy: 1.0000 - val_loss: 1.4473e-04 - val_accuracy: 1.0000
Epoch 4/5
114/114 [==============================] - 33s 291ms/step - loss: 1.2400e-04 - accuracy: 1.0000 - val_loss: 9.7494e-05 - val_accuracy: 1.0000
Epoch 5/5
114/114 [==============================] - 30s 268ms/step - loss: 8.6528e-05 - accuracy: 1.0000 - val_loss: 7.0096e-05 - val_accuracy: 1.0000


In [25]:
# Evaluate model performance on validation set
loss, accuracy = model.evaluate(X_val, y_val, verbose=1)
print(f'Validation Loss: {loss:.2f}')
print(f'Validation Accuracy: {accuracy:.2f}')

57/57 [==============================] - 1s 24ms/step - loss: 7.0096e-05 - accuracy: 1.0000
Validation Loss: 0.00
Validation Accuracy: 1.00
